## Analyze Event

Collect Event data for Metrics correlation.

In [1]:
from sqlalchemy import create_engine
from datetime import datetime
import pandas as pd
import os

today = datetime.now().strftime("%Y-%m-%d")
date = "2023-05-12"
try:
    MJ_ETL_DB = os.environ.get('MJ_ETL_DB')
    engine = create_engine(MJ_ETL_DB)
    # query
    events_df = pd.read_sql(f"SELECT * FROM events WHERE date_id = '{date}'", con=engine)
    # store csv
    events_df.to_csv('events.csv', index=False)
except BaseException as e:
    print(e)
events_df.shape

(374, 7)

In [2]:
events_df.head()

,date_id,timestamp_id,date,short_title,label,type,alert_id
0,2023-05-12,2023-05-12 00:00:44,2023-05-11 20:32:40,Web API 5xx error response is high,bot,error,115278610
1,2023-05-12,2023-05-12 00:00:44,2023-05-11 21:52:25,Discord Server Down,bot,success,118043694
2,2023-05-12,2023-05-12 00:15:44,2023-05-11 20:32:40,Web API 5xx error response is high,bot,error,115278610
3,2023-05-12,2023-05-12 00:15:44,2023-05-11 21:35:25,Discord Server Down,bot,warning,118043694
4,2023-05-12,2023-05-12 00:30:43,2023-05-11 22:04:41,Web API 5xx error response is high,bot,warning,115278610


In [20]:
group = ['short_title', 'alert_id', 'type']
events_df.groupby(group).count().reset_index()[group].to_json('events_types.json',  orient='records', indent=4)
events_df.groupby(group).count().reset_index()[group]

,short_title,alert_id,type
0,Discord Bot Exploded,110604406,error
1,Discord Bot Exploded,110604406,success
2,Discord Bot Exploded,110604406,warning
3,Discord Server Down,118043694,success
4,Discord Server Down,118043694,warning
5,There are elevated cluster errors,80840907,error
6,There are elevated cluster errors,80840907,success
7,There are elevated cluster errors,80840907,warning
8,Web API 5xx error response is high,115278610,error
9,Web API 5xx error response is high,115278610,success


In [4]:
labels = events_df['short_title'].unique()
types = events_df['type'].unique()
df = events_df.copy()
for label in labels:
    for type in types:
        df[f"{type}_{label}"] = df[['short_title', 'type']].apply(lambda x: 1 if x[0] == label and x[1] == type else 0, axis=1)
df

,date_id,timestamp_id,date,short_title,label,type,alert_id,error_Web API 5xx error response is high,success_Web API 5xx error response is high,warning_Web API 5xx error response is high,error_Discord Server Down,success_Discord Server Down,warning_Discord Server Down,error_There are elevated cluster errors,success_There are elevated cluster errors,warning_There are elevated cluster errors,error_Discord Bot Exploded,success_Discord Bot Exploded,warning_Discord Bot Exploded
0,2023-05-12,2023-05-12 00:00:44,2023-05-11 20:32:40,Web API 5xx error response is high,bot,error,115278610,1,0,0,0,0,0,0,0,0,0,0,0
1,2023-05-12,2023-05-12 00:00:44,2023-05-11 21:52:25,Discord Server Down,bot,success,118043694,0,0,0,0,1,0,0,0,0,0,0,0
2,2023-05-12,2023-05-12 00:15:44,2023-05-11 20:32:40,Web API 5xx error response is high,bot,error,115278610,1,0,0,0,0,0,0,0,0,0,0,0
3,2023-05-12,2023-05-12 00:15:44,2023-05-11 21:35:25,Discord Server Down,bot,warning,118043694,0,0,0,0,0,1,0,0,0,0,0,0
4,2023-05-12,2023-05-12 00:30:43,2023-05-11 22:04:41,Web API 5xx error response is high,bot,warning,115278610,0,0,1,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,2023-05-12,2023-05-12 23:30:44,2023-05-12 01:16:25,Discord Server Down,bot,warning,118043694,0,0,0,0,0,1,0,0,0,0,0,0
370,2023-05-12,2023-05-12 23:45:44,2023-05-12 19:35:58,There are elevated cluster errors,bot,success,80840907,0,0,0,0,0,0,0,1,0,0,0,0
371,2023-05-12,2023-05-12 23:45:44,2023-05-12 01:41:17,Discord Bot Exploded,bot,success,110604406,0,0,0,0,0,0,0,0,0,0,1,0
372,2023-05-12,2023-05-12 23:45:44,2023-05-12 19:43:40,Web API 5xx error response is high,bot,success,115278610,0,1,0,0,0,0,0,0,0,0,0,0


In [5]:
clean_df = df.drop(columns=['alert_id', 'short_title', 'type', 'label'])
clean_df.head()

,date_id,timestamp_id,date,error_Web API 5xx error response is high,success_Web API 5xx error response is high,warning_Web API 5xx error response is high,error_Discord Server Down,success_Discord Server Down,warning_Discord Server Down,error_There are elevated cluster errors,success_There are elevated cluster errors,warning_There are elevated cluster errors,error_Discord Bot Exploded,success_Discord Bot Exploded,warning_Discord Bot Exploded
0,2023-05-12,2023-05-12 00:00:44,2023-05-11 20:32:40,1,0,0,0,0,0,0,0,0,0,0,0
1,2023-05-12,2023-05-12 00:00:44,2023-05-11 21:52:25,0,0,0,0,1,0,0,0,0,0,0,0
2,2023-05-12,2023-05-12 00:15:44,2023-05-11 20:32:40,1,0,0,0,0,0,0,0,0,0,0,0
3,2023-05-12,2023-05-12 00:15:44,2023-05-11 21:35:25,0,0,0,0,0,1,0,0,0,0,0,0
4,2023-05-12,2023-05-12 00:30:43,2023-05-11 22:04:41,0,0,1,0,0,0,0,0,0,0,0,0


In [6]:
delta_seconds = (events_df['timestamp_id'] - events_df['date']).dt.seconds
delta_seconds

0      12484
1       7699
2      13384
3       9619
4       8762
       ...  
369    80059
370    14986
371    79467
372    14524
373    79159
Length: 374, dtype: int32

In [7]:
features = clean_df.columns[3:]
delta_df = clean_df.copy()
for feature in features:
    delta_df[feature] = delta_df[feature] * delta_seconds

delta_df

,date_id,timestamp_id,date,error_Web API 5xx error response is high,success_Web API 5xx error response is high,warning_Web API 5xx error response is high,error_Discord Server Down,success_Discord Server Down,warning_Discord Server Down,error_There are elevated cluster errors,success_There are elevated cluster errors,warning_There are elevated cluster errors,error_Discord Bot Exploded,success_Discord Bot Exploded,warning_Discord Bot Exploded
0,2023-05-12,2023-05-12 00:00:44,2023-05-11 20:32:40,12484,0,0,0,0,0,0,0,0,0,0,0
1,2023-05-12,2023-05-12 00:00:44,2023-05-11 21:52:25,0,0,0,0,7699,0,0,0,0,0,0,0
2,2023-05-12,2023-05-12 00:15:44,2023-05-11 20:32:40,13384,0,0,0,0,0,0,0,0,0,0,0
3,2023-05-12,2023-05-12 00:15:44,2023-05-11 21:35:25,0,0,0,0,0,9619,0,0,0,0,0,0
4,2023-05-12,2023-05-12 00:30:43,2023-05-11 22:04:41,0,0,8762,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
369,2023-05-12,2023-05-12 23:30:44,2023-05-12 01:16:25,0,0,0,0,0,80059,0,0,0,0,0,0
370,2023-05-12,2023-05-12 23:45:44,2023-05-12 19:35:58,0,0,0,0,0,0,0,14986,0,0,0,0
371,2023-05-12,2023-05-12 23:45:44,2023-05-12 01:41:17,0,0,0,0,0,0,0,0,0,0,79467,0
372,2023-05-12,2023-05-12 23:45:44,2023-05-12 19:43:40,0,14524,0,0,0,0,0,0,0,0,0,0


In [10]:
# compute index relative to time of day
SAMPLE_RESOLUTION_SECONDS = 60 * 15
day = (pd.to_datetime(delta_df['date_id']) - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
time = (delta_df['timestamp_id'] - pd.Timestamp("1970-01-01")) // pd.Timedelta('1s')
delta_df['index'] = (time - day) // SAMPLE_RESOLUTION_SECONDS
delta_df

,date_id,timestamp_id,date,error_Web API 5xx error response is high,success_Web API 5xx error response is high,warning_Web API 5xx error response is high,error_Discord Server Down,success_Discord Server Down,warning_Discord Server Down,error_There are elevated cluster errors,success_There are elevated cluster errors,warning_There are elevated cluster errors,error_Discord Bot Exploded,success_Discord Bot Exploded,warning_Discord Bot Exploded,index
0,2023-05-12,2023-05-12 00:00:44,2023-05-11 20:32:40,12484,0,0,0,0,0,0,0,0,0,0,0,0
0,2023-05-12,2023-05-12 00:00:44,2023-05-11 21:52:25,0,0,0,0,7699,0,0,0,0,0,0,0,0
1,2023-05-12,2023-05-12 00:15:44,2023-05-11 20:32:40,13384,0,0,0,0,0,0,0,0,0,0,0,1
1,2023-05-12,2023-05-12 00:15:44,2023-05-11 21:35:25,0,0,0,0,0,9619,0,0,0,0,0,0,1
2,2023-05-12,2023-05-12 00:30:43,2023-05-11 22:04:41,0,0,8762,0,0,0,0,0,0,0,0,0,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
94,2023-05-12,2023-05-12 23:30:44,2023-05-12 01:16:25,0,0,0,0,0,80059,0,0,0,0,0,0,94
95,2023-05-12,2023-05-12 23:45:44,2023-05-12 19:35:58,0,0,0,0,0,0,0,14986,0,0,0,0,95
95,2023-05-12,2023-05-12 23:45:44,2023-05-12 01:41:17,0,0,0,0,0,0,0,0,0,0,79467,0,95
95,2023-05-12,2023-05-12 23:45:44,2023-05-12 19:43:40,0,14524,0,0,0,0,0,0,0,0,0,0,95


In [11]:
features_df = delta_df[['index', 'timestamp_id', *features]].groupby('timestamp_id').max().reset_index().drop(columns=['timestamp_id'])
features_df = features_df.rename(columns={c: c.replace(' ', '_').lower() for c in features_df.columns})
features_df.to_csv('event_features.csv', index=False)
features_df

,index,error_web_api_5xx_error_response_is_high,success_web_api_5xx_error_response_is_high,warning_web_api_5xx_error_response_is_high,error_discord_server_down,success_discord_server_down,warning_discord_server_down,error_there_are_elevated_cluster_errors,success_there_are_elevated_cluster_errors,warning_there_are_elevated_cluster_errors,error_discord_bot_exploded,success_discord_bot_exploded,warning_discord_bot_exploded
0,0,12484,0,0,0,7699,0,0,0,0,0,0,0
1,1,13384,0,0,0,0,9619,0,0,0,0,0,0
2,2,0,0,8762,0,9498,0,0,0,0,0,0,0
3,3,0,0,14643,0,10399,0,0,0,0,0,0,0
4,4,0,0,10563,0,0,12319,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
91,91,0,0,11643,0,0,77359,0,0,76967,0,75867,0
92,92,77103,0,0,0,0,78259,78466,0,0,77128,0,0
93,93,78003,0,0,0,0,79159,0,0,78767,78028,0,0
94,94,14884,0,0,0,0,80059,0,0,78946,13947,0,0


In [12]:
import json

data = features_df.to_dict(orient='list')
with open('features_df.json', 'w') as outfile:
    json.dump(data, outfile)
json.dumps(data)

'{"index": [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95], "error_web_api_5xx_error_response_is_high": [12484, 13384, 0, 0, 0, 0, 1023, 603, 0, 0, 0, 0, 0, 6003, 6903, 0, 0, 0, 0, 29584, 0, 0, 0, 0, 0, 0, 0, 0, 19503, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 33003, 0, 0, 35703, 0, 0, 0, 0, 0, 0, 43323, 0, 0, 0, 45603, 0, 48723, 0, 0, 0, 0, 0, 0, 71884, 0, 0, 56403, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1323, 67203, 0, 0, 0, 5823, 6723, 0, 0, 0, 75303, 0, 77103, 78003, 14884, 0], "success_web_api_5xx_error_response_is_high": [0, 0, 0, 0, 0, 0, 0, 0, 16564, 14763, 4324, 0, 0, 0, 0, 22864, 21063, 10624, 22863, 0, 24663, 0, 29164, 0, 28263, 17824, 30063, 0, 0, 0, 0, 0, 0, 0,